##### Coursera
##### IBM Data Science Professional Certificate

### Module IX, Week III, Code Training Book
### Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

In [ ]:
#This is an instruction for myself from Coursera Course Page

To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

<b> Let's install and connect all libraries needed </b>

In [10]:
get_ipython().system(u' pip install beautifulsoup4') #beautifulsoup

     |████████████████████████████████| 122kB 5.7MB/s eta 0:00:01


In [17]:
get_ipython().system(u' pip install lxml') #lxml

     |████████████████████████████████| 5.5MB 8.5MB/s eta 0:00:01��█████▉            | 3.4MB 8.5MB/s eta 0:00:01


In [20]:
#Connect the libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

<b> Let's connect all postal codes and create a table </b>

In [21]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #I'm using the listed URL
source = requests.get(List_url).text

In [24]:
soup = BeautifulSoup(source, 'html.parser') #Implementation of BeautifulSoup

In [25]:
table = soup.find('table') #Table Creation

In [26]:
column_names = ['Postalcode','Borough','Neighborhood'] #Adding columns
df = pd.DataFrame(columns = column_names)

In [31]:
for tr_cell in table.find_all('tr'): #Adding the data
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data 

In [33]:
df.head(10) #Checking the results

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


<b> As we can see, cells with a borough that is "Not assigned" are really around. Let's delete them</b>

In [35]:
df=df[df['Borough']!='Not assigned'] #Let's delete all Not assigned labels in Borough

In [39]:
df[df['Neighborhood']=='Not assigned'] #Let's delete all Not assigned labels in Neighborhood
df.head(10) #Checkeing

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


<b>Now it's high time to combine Neighborhoods connected to the same PostalCode</b>

In [40]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [41]:
df_merge = pd.merge(df, temp_df, on='Postalcode')
df_merge.drop(['Neighborhood'], axis=1, inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods"
2,M4A,North York,"Victoria Village, Victoria Village"
4,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
6,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
8,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."


In [42]:
df_merge.shape

(103, 3)